In [48]:
import getpass
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
import chromadb
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

load_dotenv()

model = "gpt-4o-mini"

llm = ChatOpenAI(temperature=0.5, model=model, max_tokens=4096)
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
from langchain_chroma import Chroma
os.environ["LANGSMITH_PROJECT"] = "user-manual"

chroma_client = chromadb.HttpClient(host='localhost', port=8000)
print(chroma_client.heartbeat())

vector_store = Chroma(
    collection_name="user-manual",
    embedding_function=embeddings,
    client=chroma_client
)
retriever = vector_store.as_retriever()


1744678983544945093


In [49]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

template = """
**Context**: {context}
You are an expert in user manuals, your task is to provide accurate and useful information about the user manual. Use only the information provided in the documents to answer.
**User Question**: {question}

**Your Response**:
1. Provide a clear and structured answer.
2. Include examples, include prices, recommendations, or analogies if applicable.
3. Do not include any additional explanations or context.
4. Do not hallucinate.
5. Provide the page number for reference.
7. Answer the question in the language of the same question.
"""
prompt_template = ChatPromptTemplate([
    ("system", template),
    ("user", "{question}")
])

In [50]:
from langchain_core.documents import Document
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"],k=5)
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt_template.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}

graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [53]:
question = "como se tiene que lavar?, especifica presion, distancia y materiales"

In [55]:
response = graph.invoke({"question": question})
print(response["answer"])

1. **Lavado con limpiador a alta presión**:
   - **Presión**: Sigue las instrucciones del equipo de limpieza a alta presión.
   - **Distancia**: Mantén una distancia de al menos 40 cm entre el chorro de agua y la superficie del vehículo.
   - **Materiales**: No utilices un chorro que expulse agua en un chorro directo o uno que tenga un chorro rotativo. La temperatura del agua no debe exceder los 60°C.

2. **Lavado a mano**:
   - **Materiales**: Usa una esponja suave o un cepillo y productos de limpieza que no contengan solventes. Para vehículos con pintura mate, utiliza un limpiador especial para pintura mate y un paño de microfibra.
   - **Método**: Limpia el vehículo de arriba hacia abajo, primero eliminando el polvo y la suciedad gruesa, luego enjuaga con abundante agua y aplica el producto de limpieza.

Referencia: Página 356.
